In [1]:
import numpy as np
import pandas as pd
import pyreadr
import os

In [2]:
def convert_params(filename, exp, model, transform = True, save = True):
    path = f'../data/{exp}/data/'
    data = pyreadr.read_r(path+model+"_"+filename+".RData")[filename].to_numpy()
    if transform:
        data[:,0] = np.log(data[:,0])
        data[:,1] = np.log(data[:,1]/(2-data[:,1]))
    if save:
        np.save(file = path+filename+".npy", arr = data)
    else:
        return data

In [3]:
def convert_data(filename, exp, model, transform = True, save = True):
    path = f'../data/{exp}/data/'
    data = pyreadr.read_r(path+model+"_"+filename+".RData")[filename].to_numpy()
    img_shape = int(np.sqrt(data.shape[0]))
    data = np.reshape(data, newshape = (img_shape, img_shape, -1))
    if transform:
        data = np.log(data)
    if save:
        np.save(file = path+filename+".npy", arr = data)
    else:
        return data

## Convert parameters

In [13]:
convert_params("test_params", "exp_1", "brown")
convert_params("train_params", "exp_1", "brown")
convert_data("test_data", "exp_1", "brown")
convert_data("train_data", "exp_1", "brown")

In [5]:
convert_params("test_params", "exp_1", "schlather")
#convert_params("train_params", "exp_1", "brown")
convert_data("test_data", "exp_1", "schlather")
#convert_data("train_data", "exp_1", "schlather")

# Save Single model

In [2]:
exp = "exp_3"
path = f'../data/{exp}/data/'

In [3]:
for model in ["brown", "powexp"]:
    params = pyreadr.read_r(path+f"{model}_train_params.RData")["train_params"]
    data = pyreadr.read_r(path+f"{model}_train_data.RData")["train_data"].to_numpy()
    #Train/val
    train_params = params.loc[0:1999,].to_numpy()
    val_params =params.loc[2000:,].to_numpy()
    train_data =data[:,0:2000]
    train_data = np.reshape(np.swapaxes(train_data, 0,1), newshape = (-1, 25, 25))
    val_data = data[:,2000:]
    val_data = np.reshape(np.swapaxes(val_data, 0,1), newshape = (-1, 25, 25))
    #Test
    test_params = pyreadr.read_r(path+f"{model}_test_params.RData")["test_params"].to_numpy()
    test_data = pyreadr.read_r(path+f"{model}_test_data.RData")["test_data"].to_numpy()
    test_data = np.reshape(np.swapaxes(test_data, 0,1), newshape = (-1, 25, 25))
    
    np.save(path+f"{model}_train_params", train_params)
    np.save(path+f"{model}_val_params", val_params)
    np.save(path+f"{model}_train_data", train_data)
    np.save(path+f"{model}_val_data", val_data)
    np.save(path+f"{model}_test_params", test_params)
    np.save(path+f"{model}_test_data", test_data)

# Convert data / Add Schlather and brown together

In [ ]:
exp = "exp_3_1"
path = f'../data/{exp}/data/'

In [ ]:
brown_params = pyreadr.read_r(path+"brown_train_params.RData")["train_params"]
brown_data = pyreadr.read_r(path+"brown_train_data.RData")["train_data"].to_numpy()
brown_params["name"] = 0

schlather_params = pyreadr.read_r(path+"schlather_train_params.RData")["train_params"]
schlather_data = pyreadr.read_r(path+"schlather_train_data.RData")["train_data"].to_numpy()
schlather_params["name"] = 1

In [ ]:
train_params = pd.concat([brown_params.loc[0:1999,], schlather_params.loc[0:1999,]]).to_numpy()
val_params = pd.concat([brown_params.loc[2000:,], schlather_params.loc[2000:,]]).to_numpy()

In [ ]:
train_data = np.concatenate([brown_data[:,0:2000], schlather_data[:,0:2000]], axis = 1)
train_data = np.reshape(np.swapaxes(train_data, 0,1), newshape = (-1, 25, 25))
train_data.shape

(4000, 25, 25)

In [ ]:
val_data = np.concatenate([brown_data[:,2000:], schlather_data[:,2000:]], axis = 1)
val_data = np.reshape(np.swapaxes(val_data, 0,1), newshape = (-1, 25, 25))
val_data.shape

(1000, 25, 25)

In [ ]:
#Save
np.save(path+"train_params", train_params)
np.save(path+"val_params", val_params)
np.save(path+"train_data", train_data)
np.save(path+"val_data", val_data)

## Test data

In [9]:
brown_params = pyreadr.read_r(path+"brown_test_params.RData")["test_params"]
brown_data = pyreadr.read_r(path+"brown_test_data.RData")["test_data"].to_numpy()
brown_params["name"] = 0

schlather_params = pyreadr.read_r(path+"schlather_test_params.RData")["test_params"]
schlather_data = pyreadr.read_r(path+"schlather_test_data.RData")["test_data"].to_numpy()
schlather_params["name"] = 1

In [10]:
test_params = pd.concat([brown_params, schlather_params]).to_numpy()
test_params.shape

(1500, 3)

In [11]:
test_data = np.concatenate([brown_data, schlather_data], axis = 1)
test_data = np.reshape(np.swapaxes(test_data, 0,1), newshape = (-1, 25, 25))
test_data.shape

(1500, 25, 25)

In [12]:
np.save(path+"test_params", test_params)
np.save(path+"test_data", test_data)